In [19]:
import pyaudio, wave, sys
import struct
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.fftpack import fft

#open new window
%matplotlib tk

#number of audio samples per frame displayed
#4096 samples per chunk
CHUNK = 1024*4 #lower chunk size=more samples per frame, increases refresh rate
FORMAT = pyaudio.paInt16 #bytes per sample (audio format)
CHANNELS = 1 #monosound

RATE = 44100 #hz. #samples per second 

In [30]:
import random, time
from tkinter import TclError

In [37]:
# open the file for reading.
wf = wave.open('mazurka6.wav', 'rb')

# create an audio object
p = pyaudio.PyAudio()


stream=p.open(format=p.get_format_from_width(wf.getsampwidth()), 
              channels=wf.getnchannels(), rate=wf.getframerate(), input=True, 
             output=True, frames_per_buffer=1024)

In [46]:
mpl.style.use('seaborn')
sns.set(style="darkgrid")
fig, (ax, ax2) = plt.subplots(2)

x=np.arange(0, 16*CHUNK, 8) #step size 2
x_fft = np.linspace(0, RATE, CHUNK)

line, = ax.plot(x, (np.random.rand(2*CHUNK)))
#line_fft, = ax2.plot(x_fft, np.random.rand(CHUNK), '-', lw=2)
#change to log scale
line_fft, = ax2.semilogx(x_fft, np.random.rand(CHUNK), '-', lw=2)

ax.set_ylim(-100000, 100000)
ax.set_xlim(0, CHUNK)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.yaxis.set_ticklabels([])
ax.patch.set_facecolor('black')
ax.patch.set_alpha(.5) #transparency


ax2.set_xlim(20,RATE/2)
ax2.get_xaxis().set_visible(False)
ax2.yaxis.set_ticklabels([])
ax2.patch.set_facecolor('black')
ax2.patch.set_alpha(.5) #transparency

line.set_color("white")
line_fft.set_color("white")

colors = ["#330099", "#333399", "#3300CC", "#3333CC", "#333099", 
       "#333999", "#400099", "#260099"]


frame_count=0
start_time=time.time()


while True:
    data = wf.readframes(CHUNK) #read 1 chunk
    #data_int = np.array(struct.unpack(str(4*CHUNK) + 'B', data), dtype='b')[::2]+300
    data_int = np.fromstring(data, dtype=np.int16)  #frombuffer also works
    
    line.set_ydata(data_int)
    
    y_fft = fft(data_int)
    #slice and rescale
    line_fft.set_ydata(np.abs(y_fft[0:CHUNK])*2 / (10000 * CHUNK)) 
    #amplitude = 256, # of bins = CHUNK
    
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        #write to stream to play sound
        stream.write(data)
        frame_count = frame_count + 1
        
        #random background color
        ax.patch.set_facecolor(random.choice(colors))
        ax2.patch.set_facecolor(random.choice(colors))
   
    
    
    except TclError:
        frame_rate = frame_count/(time.time() - start_time)
        print("stream stopped")
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break
            


# cleanup stuff.
#stream.close()    
#p.terminate()

C:\Users\chenjf\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


stream stopped
average frame rate = 5 FPS


In [17]:
plt.plot((fft(np.sin(2 * x))))

C:\Users\chenjf\Anaconda3\lib\site-packages\numpy\core\numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
